In [ ]:
# default_exp likelihoodMethods

In [ ]:
# export
import autograd
from autograd import grad,jacobian,hessian
from autograd.scipy import stats as agss
import autograd.numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import scipy.stats as ss

from scipy.optimize import minimize

In [ ]:
def negativeLogLikelihood(x):
    def nLL(theta):
        ll = 0.0
        for i in range(len(x)):
            ll = ll + (1/len(x[i]))*(-len(x[i])/2 * np.log(2*np.pi*theta[i*2+1]**2) - (1/(2*theta[i*2+1]**2)) * np.sum((x[i] - theta[i*2])**2))
        return -1 * ll
    return nLL


In [ ]:
muStar = np.array([3.5,4,3,3,5,1,5])
sigmaStar = np.array([.5,1,1,2,2,2,2])
N = [400,200,200,100,100,100,100]
x = [np.random.normal(loc=mu,scale=sigma,size=(1,ni)) for mu,sigma,ni in zip(muStar,sigmaStar,N)]

In [ ]:
len(x)

# Autograd

In [ ]:
# theta = np.random.beta(a=1,b=1,size=(4,))
theta = np.array([0.0, 1.0,
                  0.0,1.0, 0.0,1.0,
                  0.0,1.0, 0.0,1.0, 0.0,1.0, 0.0,1.0])
gradTheta = grad(negativeLogLikelihood(x),)

maes = []
for i in range(5000):
    theta = theta - 0.001 * gradTheta(theta)
    aes = 0
    for i in range(int(len(theta)/2)):
        aes += np.abs(theta[2*i] - muStar[i])
    maes.append(aes/(len(theta)/2))

# jacobian_ = jacobian(negativeLogLikelihood(x))
# hessian_ = hessian(negativeLogLikelihood(x))
# for i in range(1000):
#     j = jacobian_(theta)
#     h = hessian_(theta)
#     theta = theta + 0.001 * np.linalg.inv(h) @ j
#     aes = np.abs(theta[0] - muStar[0]) + np.abs(theta[2] - muStar[1])
#     maes.append(aes)

In [ ]:
plt.plot(maes)

In [ ]:
maes[-1]

In [ ]:
theta[1]

In [ ]:
theta[3], theta[5]

In [ ]:
theta[7],theta[9], theta[11], theta[13]

# In terms of Bag Estimates

In [ ]:
# export
def normalizedlogLik(xi,mu,sigma):
    return (1/len(xi))*(-len(xi)/2 * np.log(2*np.pi*sigma**2) - (1/(2*sigma**2)) * np.sum((xi - mu)**2))

def getChildren(idx,N):
    if idx > N - 1:
        return np.array([idx])
    left = 2 * idx + 1
    right = left + 1
    
    return np.concatenate([getChildren(left,N),getChildren(right,N)])

def treeNegativeLogLikelihood(x,leafN):
    def LL(leafMeans,bagSigma):
        NBags = len(bagSigma)
        NInternal_Nodes = np.floor(NBags/2)
#         NLeaves = NBags - NInternal_Nodes
        ll = 0
        for idx in range(NBags):
            leafIndices = (getChildren(idx, NInternal_Nodes) - NInternal_Nodes).astype(int)
            ln = leafN[leafIndices]
            mu = np.dot(leafMeans[leafIndices],ln)/np.sum(ln)
            sigma = bagSigma[idx]
            ll = ll + normalizedlogLik(x[idx],mu,sigma)
        return -1 * ll
    return LL

### Right now I'm assuming N = $2^j$ for some j

### Generate Data

In [ ]:
N = 7
N_Internal = int(np.floor((N)/2))
NLeaves = int(N - N_Internal)
bagMuStar = np.random.normal(loc=0,scale=10,size=NLeaves)
bagN = np.random.poisson(lam=10,size=NLeaves)

X = []
for level in range(3):
    NBagsInLevel = 2**level
    start = 2**level - 1
    for bagNum in range(start,start+NBagsInLevel):
        childrenIndices = (getChildren(bagNum,N_Internal) - N_Internal).astype(int)
        childrenMus = bagMuStar[childrenIndices]
        childrenNs = bagN[childrenIndices]
        loc = np.dot(childrenMus, childrenNs) / np.sum(childrenNs)
        scale = 2**level
        X.append(np.random.normal(loc=loc,scale=scale,size=np.sum(childrenNs)))

### Initialize as local estimates

In [ ]:
mu = np.zeros(bagMuStar.shape)
sigma = np.ones(len(X))
for leafNum in range(NLeaves):
    idx = N_Internal + leafNum
    xi = X[idx]
    mu[leafNum],sigma[idx] = ss.norm.fit(xi)
    

In [ ]:
np.mean(np.abs(mu - bagMuStar))

## Run Algorithm

In [ ]:



maes = []

gradNLL_mu = grad(treeNegativeLogLikelihood(X,bagN),0)
gradNLL_sigma = grad(treeNegativeLogLikelihood(X,bagN),1)
NIter= 1000
lr = 0.1
for i in tqdm(range(NIter),total=NIter):
    if not i % 5000:
        lr = lr * .5
    deltaMu = gradNLL_mu(mu,sigma)
    deltaSigma = gradNLL_sigma(mu,sigma)
    mu = mu - lr * deltaMu
    sigma = sigma - lr * deltaSigma
    maes.append(np.mean(np.abs(mu - bagMuStar)))


In [ ]:
plt.plot(maes)